In [1]:
import datetime
import Image
import gc
import numpy as np
import os
import random
from scipy import misc
import string
import time

# Set some Theano config before initializing
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu,floatX=float32,allow_gc=False,openmp=True"
import theano

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

#import emotion_model
import dwdii_bc_model_helper as bc
import bc_models as models

random.seed(20275)

Using Theano backend.


In [2]:
print "device:", theano.config.device
print "floatX:",  theano.config.floatX
print "mode:", theano.config.mode
print "openmp:", theano.config.openmp
print "allow_gc:", theano.config.allow_gc

device: cpu
floatX: float32
mode: FAST_RUN
openmp: True
allow_gc: False


In [3]:
imagePath = "/root/bc_data/ddsm-png.25"
dataPath = "/root/bc_data/ddsm-png.25/Ddsm_png.csv"
imgResize = (150, 150)

In [4]:
os.listdir('/root/bc_data')

['ddsm', 'ddsm-hog', 'ddsm-png.25', 'ddsm-sm', 'Ddsm.csv', 'mias']

# Load Training and Test Data

In this section, the training/validation data is loaded. The load_data function pre-balances the data set by removing images from over-represented emotion classes.

In [5]:
metaData, meta2 = bc.load_training_metadata(dataPath, balanceViaRemoval=True, verbose=True)

Before Balancing
----------------
benign 3164
malignant 1096
balanaceViaRemoval.avgE: 2130
balanaceViaRemoval.theshold: 2130.0

After Balancing
----------------
benign 1613
malignant 1096


In [6]:
meta2[meta2.keys()[0]]

'0'

In [7]:
# Actually load some representative data for model experimentation
maxData = len(metaData)
X_data, Y_data = bc.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 50, imgResize=imgResize)
print X_data.shape
print Y_data.shape

Before Balancing
----------------
benign 3164
malignant 1096
balanaceViaRemoval.avgE: 2130
balanaceViaRemoval.theshold: 2130.0

After Balancing
----------------
benign 1613
malignant 1096
0.000000: A_0325_1.LEFT_MLO.LJPEG.png
0.018457: C_0192_1.RIGHT_CC.LJPEG.png
0.036914: A_0419_1.RIGHT_MLO.LJPEG.png
0.055371: A_0014_1.LEFT_CC.LJPEG.png
0.073828: C_0084_1.LEFT_CC.LJPEG.png
0.092285: B_3411_1.RIGHT_MLO.LJPEG.png
0.110742: C_0341_1.RIGHT_MLO.LJPEG.png
0.129199: B_3047_1.RIGHT_MLO.LJPEG.png
0.147656: A_0241_1.RIGHT_MLO.LJPEG.png
0.166113: A_1061_1.RIGHT_CC.LJPEG.png
0.184570: A_0201_1.RIGHT_CC.LJPEG.png
0.203027: C_0111_1.LEFT_CC.LJPEG.png
0.221484: C_0135_1.LEFT_MLO.LJPEG.png
0.239941: C_0360_1.LEFT_CC.LJPEG.png
0.258398: A_0328_1.RIGHT_CC.LJPEG.png
0.276855: A_0154_1.LEFT_MLO.LJPEG.png
0.295312: A_0602_1.RIGHT_MLO.LJPEG.png
0.313769: A_1062_1.RIGHT_CC.LJPEG.png
0.332226: A_0531_1.RIGHT_MLO.LJPEG.png
0.350683: C_0209_1.RIGHT_CC.LJPEG.png
0.369140: A_0145_1.RIGHT_CC.LJPEG.png
0.387597: B

## Split Training/Test Sets
The following code segment splits the data into training and test data sets. Currently this is a standard 80/20 split for training and test respectively after performing a random shuffle using the unison_shuffled_copies help method.

In [8]:
skippedTransforms = True
if skippedTransforms:
    X_data2 = X_data
    Y_data2 = Y_data

In [9]:
# Split the data into Training and Test sets
trainNdx = int(X_data2.shape[0] * .8)
print trainNdx

X_train, X_test = np.split(X_data2, [trainNdx])
Y_train, Y_test = np.split(Y_data2, [trainNdx])
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

2000
(2000, 150, 150)
(500, 150, 150)
(2000, 1)
(500, 1)


In [30]:
import collections
def yDist(y):
    bcCounts = collections.defaultdict(int)
    for a in range(0, y.shape[0]):
        bcCounts[y[a][0]] += 1
    return bcCounts

print "Y_train Dist: " + str(yDist(Y_train))
print "Y_test Dist: " + str(yDist(Y_test))


Y_train Dist: defaultdict(<type 'int'>, {0: 1223, 1: 777})
Y_test Dist: defaultdict(<type 'int'>, {0: 321, 1: 179})


## Define and Load Trained Model

In [31]:
# Load the bc array for our count in the model definition
bcTypes = bc.bcNumerics()
print bcTypes
print len(bcTypes)

{'benign': 0, 'malignant': 1}
2


In [32]:
# Construct the model using our help function
model = models.bc_model_v0(len(bcTypes), verbose=True, 
                                        input_shape=(1,X_train.shape[1],X_train.shape[2]))

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_1 (Convolution2D)    (None, 32, 143, 143)2080        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 32, 143, 143)0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)      (None, 32, 71, 71)  0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)    (None, 32, 67, 67)  25632       maxpooling2d_1[0][0]             
___________________________________________________________________________________________

## Training the Model

The following code segment trains the model using the run_network helper function. 

In [33]:
loadWeights = False
if loadWeights:
    model.load_weights("dwdii-bc-150-v0-Cloud.hdf5")

In [34]:
# Reshape to the appropriate shape for the CNN input
testX = X_test.reshape(X_test.shape[0], 1, X_train.shape[1],X_train.shape[2])
trainX = X_train.reshape(X_train.shape[0], 1, X_train.shape[1],X_train.shape[2])

In [35]:
print "Training start: " + str(datetime.datetime.now())
m, h = models.run_network([trainX, testX, Y_train, Y_test], model, batch=50, epochs=30, verbosity=1)

Training start: 2017-03-21 00:54:56.350493
(2000, 2)
(500, 2)
Training model...
Train on 2000 samples, validate on 500 samples
Epoch 1/30
2000/2000 [==============================] - 476s - loss: 0.7032 - acc: 0.6345 - val_loss: 0.5791 - val_acc: 0.6860
Epoch 2/30
2000/2000 [==============================] - 474s - loss: 0.5904 - acc: 0.6745 - val_loss: 0.6132 - val_acc: 0.6420
Epoch 3/30
2000/2000 [==============================] - 490s - loss: 0.5820 - acc: 0.6655 - val_loss: 0.5992 - val_acc: 0.6640
Epoch 4/30
2000/2000 [==============================] - 474s - loss: 0.5621 - acc: 0.6825 - val_loss: 0.5565 - val_acc: 0.6740
Epoch 5/30
2000/2000 [==============================] - 475s - loss: 0.5437 - acc: 0.7055 - val_loss: 0.5496 - val_acc: 0.6860
Epoch 6/30
2000/2000 [==============================] - 474s - loss: 0.5205 - acc: 0.7225 - val_loss: 0.5437 - val_acc: 0.6860
Epoch 7/30
2000/2000 [==============================] - 517s - loss: 0.4922 - acc: 0.7530 - val_loss: 0.5620 - 

In [ ]:
model.save_weights("dwdii-bc-150-v0-Cloud.hdf5", overwrite=True)

### Experiment Results

* bc_model_v0 (150x150, 800/200): 182s - loss: 0.0560 - acc: 0.9813 - val_loss: 1.9918 - val_acc: 0.6800
* bc_model_v0 (150x150, 2000/500): 473s - loss: 0.0288 - acc: 0.9925 - val_loss: 1.4040 - val_acc: 0.7260
   * somewhat balanced, Y_train Dist {0: 1223, 1: 777}, Y_test Dist: {0: 321, 1: 179}